In [1]:
import keras
from keras.utils import to_categorical
import pickle
from sklearn import preprocessing
import random
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from os.path import join
from sklearn.model_selection import train_test_split

/Users/md705/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [23]:
def setting():
    info = {
        'data_path' : "dataset_training/video_phases_background_removed.pickle",
        "model_path": "model"
    }
    return info

def loaddata(filepath):
    with open(filepath, 'rb') as file:
        data = pickle.load(file)
    return data

def vector_cat(label_name, le):
    vec = np.zeros(len(list(le.classes_)))
    vec[int(le.transform([label_name]))] = 1
    return vec

def shuffle(vec1, vec2):
    c = list(zip(vec1,vec2))
    random.shuffle(c)
    vec1[:], vec2[:] = zip(*c)
    return vec1, vec2

def set_x_y(data):
    
    labels = list(data.keys())
    le = preprocessing.LabelEncoder()
    le.fit(labels)
    x = []
    y = []
    for key in data.keys():
        for img in data[key]:
            x.append(img.flatten())
            y.append(vector_cat(key, le))
    return x,y, le

def train_test_shuffle(x,y):
    x = np.array(x)
    y = np.array(y)
    train_x, test_x, train_y, test_y = train_test_split(x,y,test_size=0.33, random_state=None)    
    return train_x, train_y, test_x, test_y    
    

In [24]:
# main

## Setting
info = setting()
data = loaddata(info['data_path'])
x, y, le = set_x_y(data)
train_x, train_y, test_x, test_y = train_test_shuffle(x,y)

In [29]:
## Learning
size_input = train_x[0].shape[0]
model = Sequential()
model.add(Dense(100, input_dim=size_input, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(len(le.classes_), activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(train_x, train_y, epochs=5, batch_size=10)


# evaluate the model
scores = model.evaluate(train_x, train_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# 10. Evaluate model on test data
score = model.evaluate(test_x, test_y, verbose=0)

Epoch 1/5
7562/7562 [==============================] - 40s 5ms/step - loss: 3.2293 - acc: 0.7996
Epoch 2/5
7562/7562 [==============================] - 39s 5ms/step - loss: 3.2236 - acc: 0.8000
Epoch 3/5
7562/7562 [==============================] - 39s 5ms/step - loss: 3.2236 - acc: 0.8000
Epoch 4/5
7562/7562 [==============================] - 40s 5ms/step - loss: 3.2236 - acc: 0.8000
Epoch 5/5
7562/7562 [==============================] - 3s 346us/step

acc: 80.00%


In [30]:
# Save Model
model.save(join('model','seq_model.h5'))
with open(join('model','seq_model_encoder.pickle'),'wb') as file:
    pickle.dump(le, file)

with open(join('model','seq_model_train_x.pickle'),'wb') as file:
    pickle.dump(train_x, file)

with open(join('model','seq_model_train_y.pickle'),'wb') as file:
    pickle.dump(train_y, file)

with open(join('model','seq_model_test_x.pickle'),'wb') as file:
    pickle.dump(test_x, file)

with open(join('model','seq_model_test_y.pickle'),'wb') as file:
    pickle.dump(test_y, file)

In [31]:
model.evaluate(train_x, train_y)

7562/7562 [==============================] - 3s 333us/step


[3.223618984222412, 0.800000011920929]

In [32]:
model.evaluate(test_x, test_y, verbose=0)

[3.2236189851182413, 0.8000000121448863]

In [37]:
np.array_equal(test_x, test_y)

False

In [47]:
model.predict_proba(np.array([test_x[100]]))

array([[0., 0., 0., 0., 0.]], dtype=float32)

In [42]:
test_y[1]

array([0., 1., 0., 0., 0.])